In [ ]:
DATA_DIR = '../datamount'

In [ ]:
cd {DATA_DIR}

In [1]:
#place your eternafold params
#see https://github.com/eternagame/EternaFold for more details
%env ETERNAFOLD_PATH=/home/hoyso/miniconda3/envs/hoyso_ml/bin/eternafold-bin
%env ETERNAFOLD_PARAMETERS=/home/hoyso/miniconda3/envs/hoyso_ml/lib/eternafold-lib/parameters/EternaFoldParams.v1

env: ETERNAFOLD_PATH=/home/hoyso/miniconda3/envs/hoyso_ml/bin/eternafold-bin
env: ETERNAFOLD_PARAMETERS=/home/hoyso/miniconda3/envs/hoyso_ml/lib/eternafold-lib/parameters/EternaFoldParams.v1


In [2]:
from arnie.mfe import mfe
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm

/tmp/ipykernel_903159/3938609067.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
import shutil
from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing import cpu_count

def split_dataframe(df, chunk_size = 10000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks
    
def mfe_eternafold(seqs):
    result = []
    
    for x in tqdm(seqs):
        result.append(mfe(x,package="eternafold"))
    return result

In [4]:
train_df = pd.read_parquet('train_data_new.parquet')

result = Parallel(n_jobs=cpu_count())(
    delayed(mfe_eternafold)(x)
    for i,x in enumerate(split_dataframe(train_df.sequence, 10000))
)
train_df['mfe_eternafold'] = np.concatenate(result, axis=0)
train_df.to_parquet('train_data_new.parquet')
del train_df

100%|██████████| 10000/10000 [09:06<00:00, 18.29it/s]


In [5]:
test_df = pd.read_parquet('test_sequences_new.parquet')

result = Parallel(n_jobs=cpu_count())(
    delayed(mfe_eternafold)(x)
    for i,x in enumerate(split_dataframe(test_df.sequence, 10000))
)
test_df['mfe_eternafold'] = np.concatenate(result, axis=0)
test_df.to_parquet('test_sequences_new.parquet')
del test_df

100%|██████████| 3823/3823 [31:09<00:00,  2.04it/s]
